In [2]:
from openparticle import *

$H(0) = H_f + H_{fb} + H_Y + H_+ + X_\Delta$

In 1 + 1D, 

$H_f = \sum_\sigma \int_p \frac{m^2}{p^+}b_p^\dagger \rightarrow \frac{1}{2L}\sum_\sigma \sum_n \frac{m^2}{p^+(n)}b_p^\dagger$

$H_{fb} = \sum_\sigma \int_{p,k} \left(\frac{m^2}{p^+} +\frac{\mu^2}{k^+}\right)b_p^\dagger a_k^\dagger \rightarrow \frac{1}{2L}\sum_\sigma \sum_{n,m} \left(\frac{m^2}{p^+(n)} +\frac{\mu^2}{k^+(m)}\right)b_p^\dagger a_k^\dagger$

$H_Y = g\sum_{\sigma_1, \sigma_2} \int_{p_1, p_2, k}\theta(\Delta^2) \delta(p_1 - p_2 - k) [\bar{u}_2u_1 b_2^\dagger b_1 a_0^\dagger + \bar{u}_1 u_2 b_1^\dagger b_2 a_0]$


$H_+ = g^2 \sum_{\sigma_1, \sigma_2} \int_{p_1, p_2, k_1, k_2} \theta(\Delta^2) \delta(p_1 + k_1 - p_2 - k_2) \bar{u}_2\frac{\gamma^+}{2(p_1^+ + k_1^+)}u_1$

$\bar{u}_{\sigma'}(p')u_\sigma(p) = m\sqrt{p^+ p^{\prime +}} \left(\frac{1}{p^+} + \frac{1}{p^{\prime +}} \right)$

$\bar{u}_{\sigma'}(p')\gamma^+ u_\sigma(p) = 2\sqrt{p^+ p^{\prime +}}\delta_{\sigma', \sigma}$

In [15]:
m, mu = 1, 0.01
L = 1e4
Lambda = 3

In [16]:
def pplus(k):
    return 2 * np.pi * k / L

In [17]:
Hf = ParticleOperator({})
for spin in [-1, 1]:
    for mode_n, n in enumerate(np.arange(0.5, Lambda, 0.5)):
        hf = m**2 / pplus(n)
        Hf += hf * ParticleOperator('b' + str(mode_n) + "^")
Hf = 1/(2*L) * Hf
Hf

0.3183098861837907 * b0^
0.15915494309189535 * b1^
0.1061032953945969 * b2^
0.07957747154594767 * b3^
0.06366197723675814 * b4^